This is the Python implementation of the 3D-RFT Code based on the framework
proposed by Agarwal et al. --> https://doi.org/10.1073/pnas.2214017120

In [1]:
######## IMPORTS ########
import math

import numpy as np
import scipy as sp
from mpl_toolkits import mplot3d
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
import pandas as pd

In [2]:
from src import rft_functions

In [3]:
######## INPUTS ########

## Model selection
MODEL = "CylinderNormal"

## Physical properties
BULK_DENSITY = 1520
FRICTION_MATERIAL_DEG = 20
FRICTION_MATERIAL = math.tan(np.deg2rad(FRICTION_MATERIAL_DEG))
FRICTION_SURFACE = 0.4
GRAVITY = 9.81
MATERIAL_CONSTANT = (
    GRAVITY
    * BULK_DENSITY
    * (
        894 * FRICTION_MATERIAL**3
        - 386 * FRICTION_MATERIAL**2
        + 89 * FRICTION_MATERIAL
    )
)

## Movement defintions
LINEAR_VELOCITY = 0.1
DIR_ANGLE_XZ_DEG = -90
DIR_ANGLE_Y_DEG = -90

ROTATION = True
ANGULAR_VELOCITY = np.array([0, 0, -2 * sp.pi])

## Depth parameters in mm
DEPTH = 100

## Miscellaneous & Plots
THRESHOLD = 1.0e-12

In [4]:
## STL Processing
(
    point_list,
    normal_list,
    area_list,
    depth_list,
    object_width_x,
    object_width_y,
    object_height,
    trg,
) = rft_functions.import_mesh(MODEL)

In [12]:
figure = plt.figure(figsize=(10, 10))
axes = mplot3d.Axes3D(figure)

axes.add_collection3d(mplot3d.art3d.Poly3DCollection(trg.vectors))
scale = trg.points.flatten(-1)
axes.auto_scale_xyz(scale, scale, scale)
axes.set_xlabel('X')
axes.set_ylabel('Y')
axes.set_zlabel('Z')
axes.set_xlim3d(-object_width_x, object_width_x)
axes.set_ylim3d(-object_width_y, object_width_y)
axes.set_zlim3d(0, object_height)


plt.show()

TypeError: order must be str, not int

<Figure size 1000x1000 with 0 Axes>

In [11]:
print(trg.vectors)

[[[-24.95691  -70.        26.46719 ]
  [-25.       -70.        25.      ]
  [-23.437838 -70.        25.      ]]

 [[-23.437838 -70.        25.      ]
  [-25.       -70.        25.      ]
  [-24.95691  -70.        23.53281 ]]

 [[-23.437838 -70.        25.      ]
  [-24.95691  -70.        23.53281 ]
  [-23.437838 -70.        23.437668]]

 ...

 [[ 21.864899  70.        35.93632 ]
  [ 21.864899  70.        34.37399 ]
  [ 20.302734  70.        35.93632 ]]

 [[ 21.864899  70.        14.063679]
  [ 20.302734  70.        14.063679]
  [ 20.302734  70.        15.62601 ]]

 [[ 21.864899  70.        14.063679]
  [ 20.302734  70.        15.62601 ]
  [ 21.864899  70.        15.62601 ]]]
